# TensorRT Engine Report Card

Use this Jupyter worksheet to get a quick overview of the structure and characteristics of a TensorRT Engine plan.

## Load JSON Files

In [ ]:
import IPython
from ipywidgets import widgets
from trex import *

# Choose an engine file to load.
engine_name = "../tests/inputs/mobilenet.qat.onnx.engine"
engine_name = "../tests/inputs/mobilenet_v2_residuals.qat.onnx.engine"
set_wide_display()

# Execute the cell, then press the Select button, choose an engine file, and move to the next cell.
rootdir = '/data/onnx/'
fc = display_filechooser(rootdir)

In [ ]:
if fc.selected is not None:
    engine_name = fc.selected
    
assert engine_name is not None
plan = EnginePlan(f"{engine_name}.graph.json", f"{engine_name}.profile.json", f"{engine_name}.metadata.json")

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Plan Summary</div></html>

In [ ]:
plan.summary()

In [ ]:
report_card_table_view(plan);

### Timings

In [ ]:
def plot_engine_timings(timing_json_file: str):
    """Plot the engine profiling timings"""
    import json
    try:
        with open(timing_json_file) as json_file:
            data = json.load(json_file)
    except:
        raise ValueError(f"Could not load JSON file {timing_json_file}")

    y = [rec['latencyMs'] for rec in data]
    x = range(len(data))

    fig = px.scatter(x=x, y=y)

    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

    fig = px.histogram(x=y, histfunc="count", nbins=len(x)//10)
    fig.show()

plot_engine_timings(timing_json_file= f"{engine_name}.timing.json")

## Performance

In [ ]:
report_card_perf_overview(plan);

## Memory Footprint

In [ ]:
report_card_memory_footprint(plan);

## Graph Rendering

Choose how to color nodes.

In [ ]:
report_card_draw_plan_graph(plan, engine_name)

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Convolutions</div></html>

In [ ]:
convs = plan.get_layers_by_type('Convolution')
report_card_convolutions_overview(convs)

## Tactics

In [ ]:
latency_vs_prec_per_conv = partial(
    plotly_bar2,
    convs,
    values_col='latency.pct_time',
    names_col='Name',
    color='tactic')

latency_vs_prec_per_conv("Latency per Layer (color=Tactics)")

tactic_cnt = group_count(plan.df, 'tactic')
display_df(tactic_cnt)

## Experimental 

The data below are based on partial information and are provided here just for fun. Don't take it too seriously.

Examples of problematic assumptions:
* Convolutions and matrix-multiplications are implemented using implicit-gemm. In practice, various algorithms might be used, which will affect the arithmetic-intensity.
* Input/output activations (feature-maps) and parameters (weights and other constants) are read once, and from device global memory. In practice, it is likely that activations will be resident in the L2 cache, which has a much higher bandwidth compared to global memory. Matrix multiplication is performed using activation tiles which are read (reused) multiple times. These reads are ignored in the calculation of the arithmetic-intensity.
* Compute-efficiency and memory-efficiency are calculated by dividing the MACs or bytes by the total time per layer, instead of dividing by the time spent only on compute or only on memory access.
* Some convolutions have fued operations (e.g. SiLU activation) which are currently ignored in the calculation of the number of operations.


Performance optimization references:
* [Nvidia DL Peformance Background: Understand Performance](https://docs.nvidia.com/deeplearning/performance/dl-performance-gpu-background/index.html#understand-perf)
* [Nvidia DL Peformance: GEMM](https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#math-mem)
* [CUDA Programming: Instruction Throughput](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#maximize-instruction-throughput)
* [TensorRT Developer Guide: Performance Best Practices](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#performance)

In [ ]:
import plotly.express as px

def report_card_gemm_MNK(plan: pd.DataFrame):
    def render_scatter3d(choice, x, y, z, color, size):
        convs = plan.get_layers_by_type('Convolution')
        fig = px.scatter_3d(convs, x=x, y=y, z=z, color=color, size=size, 
                size_max=18, symbol='precision', opacity=0.7)

        fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
        fig.show()

    dropdown_choices = {
        "MxNxK color=mean time; size=mean time": ('attr.M', 'attr.N', 'attr.K', 'latency.avg_time', 'latency.avg_time',),
        "MxNxK color=arithmetic intensity; size=mean time": ('attr.M', 'attr.N', 'attr.K', 'attr.arithmetic_intensity', 'latency.avg_time',),
        "MxNxK color=compute efficiency; size=mean time": ('attr.M', 'attr.N', 'attr.K', 'attr.compute_efficiency', 'latency.avg_time',),
        "MxNxK color=memory efficiency; size=mean time": ('attr.M', 'attr.N', 'attr.K', 'attr.memory_efficiency', 'latency.avg_time',),
    }

    InteractiveDiagram(render_scatter3d, dropdown_choices, 'Diagram')
    
report_card_gemm_MNK(plan);

In [ ]:
fig = px.scatter_3d(convs, x='attr.compute_efficiency', y='attr.memory_efficiency', z='latency.avg_time',
    color='total_footprint_bytes', size='latency.avg_time', size_max=18,
    symbol='precision',
    opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
def report_card_perf_scatter(plan: pd.DataFrame):
    def render_scatter(choice, x, y, color, size):
        convs = plan.get_layers_by_type('Convolution')
        fig = px.scatter(convs, x=x, y=y, color=color, size=size, 
                size_max=18, symbol='precision', opacity=0.7)

        fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
        fig.show()

    dropdown_choices = {
        "Compute-efficiency vs. memory-efficiency": (
            'attr.compute_efficiency', 'attr.memory_efficiency', 'total_footprint_bytes', 'attr.macs',),
        "Memory footprint vs FMAs": (
            'total_footprint_bytes', 'attr.macs', 'attr.compute_efficiency', 'latency.avg_time',),
        "Arithmetic vs. compute-efficiency": (
            'attr.arithmetic_intensity', 'attr.compute_efficiency', 'attr.memory_efficiency', 'latency.avg_time',),
    }

    InteractiveDiagram(render_scatter, dropdown_choices, 'Diagram')
    
report_card_perf_scatter(plan);

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Layer Lint Utility</div></html>

Linting functions perform static analysis of the plan to flag possible performance hazards.<br>
See TensorRT's [Performance Best Prcatices](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#performance) for more information.

<b>The linting is in an early experimental stage and may be imcomplete or erronous.<b>

## Convolution Lint

Ideally all Float16 and INT8 convolutions are accelerated on Tensor Cores, so `ConvLinter` uses heuristics on the kernel name to determine if the kernel is accelerated on TCs.  This method is probably incorrect is some cases and needs more investment.

In [ ]:
linter = ConvLinter(plan)
display_df(linter.lint())

## Reformat Lint

Reformat layers copy their input to their output while making some changes to the data.<br>
A Reformat layer may change the data layout (e.g. from NCHW to NC32HW) or perform data-type conversion (e.g. from float32 to INT8).

A Reformat layer is added by to the engine graph by TensorRT's graph-optimizer for one of several reasons, which are indicated by the `attr.origin` field.
* REFORMAT: type or layout conversion.
* SLICE: slice layer output conversion.
* CONCAT: concat layer input conversion.

Reformat layers that perform data-type conversion from float32/float16 to INT8, or vice-versa, may indicate poorly placed Q/DQ layers in a QAT network. <br>
These are Q/DQ layers which could not be fused with another layer in the engine graph and may be quite costly in latency.

In [ ]:
display_df(ReformatLinter(plan).lint())

## Slice Linter

Slice layers that perform data-type conversion may indicate an optimization opportunity.

In [ ]:
display_df(SliceLinter(plan).lint())

## Q/DQ Linter

Quantize/Dequenatize layers perform a copy with quantization/dequantization.<br>
Unfused Q/DQ layers ("dangling Q/DQ") are very wasteful and usually indicate poorly placement of fake-quantization in the training model.

In [ ]:
display_df(QDQLinter(plan).lint())

## Pointwise Layers

Pointwise and Elementwise layers can be fused to create larger kernels.<br>
Here you can explore how well these layers managed to fuse.

In [ ]:
report_card_pointwise_lint(plan)